In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import string as strings
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/spotify_millsongdata.csv")

In [5]:
df.head(10)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,"Well, you hoot and you holler and you make me ..."
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong \r\nYou're e..."
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,I was out with the morning sun \r\nCouldn't s...
9,ABBA,Crying Over You,/a/abba/crying+over+you_20177611.html,I'm waitin' for you baby \r\nI'm sitting all ...


In [6]:
df=df.sample(5000).drop('link',axis =1).reset_index(drop='TRUE')

In [7]:
df.tail(5)

,artist,song,text
4995,Miley Cyrus,1 In Amillon,How did I get here? I turned around and there ...
4996,The Monkees,Time And Time Again,Time and time again \r\nNaughty girls get me ...
4997,Gary Valenciano,Take Me Out Of The Dark,Just what is it in me? \r\nSometimes I just d...
4998,Barbra Streisand,Avinu Malkeinu,Hear our prayer \r\nWe have sinned before The...
4999,Uriah Heep,Hold Your Head Up,And if it's bad \r\nDon't let it get you down...


In [8]:
#preprocessing
#convert text to lower case and replace /r /n /no.s
df['text'] = df['text'].str.lower().replace(regex=r'^\w\s', value='').replace(regex=r'\n', value=' ').replace(regex=r'\r', value=' ')
# Remove punctuation in a single line
df['text'] = df['text'].str.replace('[{}]'.format(strings.punctuation), '')
df['text']

<ipython-input-8-b7b7781ff60f>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[{}]'.format(strings.punctuation), '')


0       im drunk again excuse the hiccup    the rednec...
1       could have stood to watch you walk away    and...
2       here by the sea and sand    nothing ever goes ...
3       thats how much i love you    thats how much i ...
4       oh i was moved by your screen dream    cellulo...
                              ...                        
4995    how did i get here i turned around and there y...
4996    time and time again    naughty girls get me in...
4997    just what is it in me    sometimes i just dont...
4998    hear our prayer    we have sinned before thee ...
4999    and if its bad    dont let it get you down    ...
Name: text, Length: 5000, dtype: object

In [9]:
df

,artist,song,text
0,Kid Rock,Hillbilly Stomp,im drunk again excuse the hiccup the rednec...
1,Chris Rea,Love Turns To Lies,could have stood to watch you walk away and...
2,Who,Sea And Sand,here by the sea and sand nothing ever goes ...
3,Rihanna,Hate That I Love You,thats how much i love you thats how much i ...
4,Roxy Music,2Hb,oh i was moved by your screen dream cellulo...
...,...,...,...
4995,Miley Cyrus,1 In Amillon,how did i get here i turned around and there y...
4996,The Monkees,Time And Time Again,time and time again naughty girls get me in...
4997,Gary Valenciano,Take Me Out Of The Dark,just what is it in me sometimes i just dont...
4998,Barbra Streisand,Avinu Malkeinu,hear our prayer we have sinned before thee ...


In [10]:
#tokenization using porter stemmer
stemmer=PorterStemmer()
def token(txt):
  token=nltk.word_tokenize(txt)
  a=[stemmer.stem(w) for w in token]
  return " ".join(a)
  #test


In [11]:
#tokenization working -
token("you're really beautiful")

"you 're realli beauti"

In [12]:
#apply to whole text column
df['text']=df['text'].apply(lambda x: token(x))

In [13]:
df['text']

0       im drunk again excus the hiccup the redneck or...
1       could have stood to watch you walk away and pu...
2       here by the sea and sand noth ever goe as plan...
3       that how much i love you that how much i need ...
4       oh i wa move by your screen dream celluloid pi...
                              ...                        
4995    how did i get here i turn around and there you...
4996    time and time again naughti girl get me in tro...
4997    just what is it in me sometim i just dont know...
4998    hear our prayer we have sin befor thee have co...
4999    and if it bad dont let it get you down you can...
Name: text, Length: 5000, dtype: object

In [14]:
tfid=TfidfVectorizer(analyzer='word',stop_words='english')
matrix=tfid.fit_transform(df['text'])

In [15]:
similar =cosine_similarity(matrix)

In [16]:
def recommender(song_name):
    try:
        idx = df[df['song'] == song_name].index[0]
    except IndexError:
        print(f"The song '{song_name}' does not exist in the dataset.")
        return []

    distance = sorted(list(enumerate(similar[idx])), reverse=True, key=lambda x: x[1])
    songs = []
    for s_id in distance[1:5]:
        songs.append(df.iloc[s_id[0]]['song'])
    return songs

In [17]:
#recoomender working on a sample input song
recommender("King Nothing")

The song 'King Nothing' does not exist in the dataset.


[]

In [18]:
pickle.dump(similar, open("similarity","wb"))

In [19]:
pickle.dump(df, open("df","wb"))